#### El Parque Nacional Guatopo
**Autor:** Javier Martínez

**Tutor:** Isabel LLatas

Con GEE es posible acceder a una colección de imágenes (o a una imágen) según el catálogo de productos disponibles en [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/).

Para el siguiente ejemplo se ha seleccionado al *Parque Nacional Guatopo*. Por lo que;

1. Se importan los módulos necesarios.
2. Se da inicio a la API.
3. Se consulta la colección de imágenes [*WCMC/WDPA/current/polygons*](https://developers.google.com/earth-engine/datasets/catalog/WCMC_WDPA_current_polygons), seleccionando la imágen *Guatopo*.
4. Se crea el mapa del *Parque Nacional Guatopo*.

 
Se recomienda usar [Jupyter nbview](https://nbviewer.jupyter.org/) para la visualización y estudio del *Notebook*.


In [107]:
# Módulos
import ee
import folium
import geehydro

In [109]:
# Inicio de API
ee.Initialize()

In [111]:
polygonCollection = 'WCMC/WDPA/current/polygons' # Colección
polygonName = 'Guatopo' # Imágen
# Consultando la colección y seleccionando la  imagen Guatopo
polygon = ee.FeatureCollection(polygonCollection) \
            .filter(ee.Filter.eq('NAME', polygonName))

# centroide del poligono
centroide = polygon\
                .limit(1)\
                .geometry()\
                .centroid()\
                .getInfo()\
                .get('coordinates',[-66.8600685,10.5165848,])[::-1]

Una vez tomada la imágen se procede a crear un mapa con la ayuda de *folium*. Se ha seleccionado la localización $10°30'59.7"N 66°51'36.3"W$ para el inicio del mapa con un zoom de 9, además se incorpora el control de escala y el control de layers. 

In [112]:
# Mapa
Map = folium.Map(location=centroide,zoom_start=11,control_scale =True,width='100%',height='100%')
Map.addLayer(polygon,name=polygonName) # Agregando polígono
Map.setControlVisibility(layerControl=True,fullscreenControl=True,latLngPopup=True) # Layer control
Map

Para Guardar el mapa en formato .HTML;

> Map.save('mapa_guatopo.html')

In [113]:
# determinando poligono
features = polygon\
                .limit(1)\
                .getInfo()\
                .get('features',[{}])[0]
                
# coordenadas del poligono               
coordinates = polygon.geometry().coordinates().getInfo()
# propiedades del poligono
properties = features.get('properties',{})


# area en km cuadrados
print(properties.get('GIS_AREA'))

# area en kilometros cuadrados
print(0.01*polygon.geometry().area().divide(100*100).getInfo())
# area en metros
print(polygon.geometry().area().getInfo())


1223.46692022336
1223.4704597852344
1223470459.7852345


In [114]:

def eeGrid(x,y):
    """
    Función para la creación de grillas con ee
    x = vector latitud
    y = 
    """

    def feat(xcor, y_vector):
        return ee.FeatureCollection( y_vector.map(lambda ycor: ee.Feature( ee.Geometry.Point([xcor,ycor]) )) )

    return ee.FeatureCollection(x.map(lambda xcor: feat(xcor, y_vector=y) )).flatten()



x = list(map(lambda x: x[0],coordinates[0]))
y = list(map(lambda x: x[1],coordinates[0]))


x = ee.List.sequence(min(x),max(x)+0.00008999,0.00008999)
y = ee.List.sequence(min(y),max(y)+0.00008999,0.00008999)


# ee.Geometry.Point(y.getInfo()[0],x.getInfo()[0]).distance(
#         ee.Geometry.Point(y.getInfo()[0],x.getInfo()[1]),1).getInfo()

# Map = folium.Map()
# Map.addLayer(eeGrid(x,y))
# Map


In [115]:

# MODIS MOD13Q1
modis = ee.ImageCollection('MODIS/006/MOD13Q1')
modis = modis.filterDate(ee.DateRange('2016-01-01','2019-12-01'))

In [116]:
# select EVI and NDVI
evi = modis.select('EVI')
ndvi = modis.select('NDVI')

In [117]:
def scale_factor(image):
    # scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image,['system:time_start'])
# mapping function to multiply by the scale factor
scaled_evi = evi.map(scale_factor)
scaled_ndvi = ndvi.map(scale_factor)

In [118]:
# mean NDVI in the Xingu Park
Map = folium.Map(location=centroide,zoom_start=11,control_scale =True,width='100%',height='100%')
Map.addLayer(scaled_ndvi.mean().clip(polygon),name=polygonName,                  vis_params={'min': 0,
                              'max': 1,
                              'palette': ['red', 'yellow','green']}) # Agregando polígono
Map.setControlVisibility(layerControl=True,fullscreenControl=True,latLngPopup=True) # Layer control
Map

In [119]:
imagen = scaled_ndvi.mean().clip(polygon)
type(imagen)

ee.image.Image

In [120]:
lonLat = imagen.pixelCoordinates(imagen.projection())

In [121]:
x = lonLat.select('x').multiply(10000000).toInt()
y = lonLat.select('y').multiply(10000000).toInt()

type(x)

# Map = folium.Map()
# Map.addLayer(eeGrid(x,y))
# Map

ee.image.Image

In [122]:
ee.Image.pixelLonLat().projection().getInfo()

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}

In [123]:
latlon = ee.Image.pixelLonLat()


coords = latlon.select(['longitude', 'latitude'])\
                .reduceRegion(**{
                                'reducer': ee.Reducer.toList(),
                                'geometry': polygon,
                                'scale': 5000
                })

lat = ee.List(coords.get('latitude'))
lon = ee.List(coords.get('longitude'))


point_list = lon.zip(lat)
multiPoint = ee.Geometry.MultiPoint(point_list)

Map = folium.Map(location=centroide)
Map.addLayer(multiPoint)
Map